In [3]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


### *Connection SQL Server*

In [44]:
import pyodbc
import pandas as pd

In [5]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER = server, DATABASE = database, UID = user, PWD = password)
        print("MS SQL Server Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return conn

In [3]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'VAIO'
database = 'master'
user = 'sa'
password = '14531453'

In [12]:
conn = create_server_connection(driver, server, database, user, password) 

MS SQL Server Database connection successful


In [14]:
conn.autocommit = True

In [15]:
csr = conn.cursor()

In [8]:
create_db_query = 'CREATE DATABASE Lab_DB'

In [16]:
csr.execute(create_db_query)

In [19]:
csr.execute('USE Lab_DB')

In [20]:
query = 'CREATE TABLE Lab_Table( \
        ID int IDENTITY(1,1) NOT NULL,\
        FirstName varchar(255) NOT NULL,\
        LastName varchar(255) NOT NULL,\
        PRIMARY KEY(ID))'

In [21]:
csr.execute(query)

In [22]:
csr.execute("INSERT Lab_Table (FirstName, LastName) VALUES (?, ?)", 'Tom', 'Cat')
csr.execute("INSERT Lab_Table (FirstName, LastName) VALUES (?, ?)", 'Jerry', 'Mouse')
csr.execute("INSERT Lab_Table (FirstName, LastName) VALUES (?, ?)", 'Stefan', 'Müller')
csr.execute("INSERT Lab_Table (FirstName, LastName) VALUES (?, ?)", 'Dennis', 'Bergkamp')
csr.execute("INSERT Lab_Table (FirstName, LastName) VALUES (?, ?)", 'Tom', 'Hanks')
csr.execute("INSERT Lab_Table (FirstName, LastName) VALUES (?, ?)", 'Bart', 'Simpson')
csr.execute("INSERT Lab_Table (FirstName, LastName) VALUES (?, ?)", 'Bruce', 'Lee')
csr.execute("INSERT Lab_Table (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [23]:
csr.execute("DELETE from Lab_Table where FirstName= ?", 'Stefan')

In [24]:
csr.rowcount

1

In [25]:
csr.execute("DELETE from Lab_Table where FirstName= ?", 'Tom')

In [26]:
csr.rowcount

2

In [27]:
csr.execute('SELECT * FROM [dbo].[Lab_Table]').fetchall()

[(2, 'Jerry', 'Mouse'),
 (4, 'Dennis', 'Bergkamp'),
 (6, 'Bart', 'Simpson'),
 (7, 'Bruce', 'Lee'),
 (8, 'Bob', 'Marley')]

In [28]:
csr.rowcount

-1

In [29]:
csr.execute('SELECT * FROM [dbo].[Lab_Table]').fetchone()

(2, 'Jerry', 'Mouse')

In [30]:
csr.rowcount # rowcount okuma işlemlerinde rowcount bilgisi değişmez.

-1

In [31]:
csr.execute('SELECT * FROM [dbo].[Lab_Table]').fetchval()

2

In [33]:
csr.execute('SELECT FirstName, LastName FROM Lab_Table')
while True:
    row = csr.fetchone()
    if not row:
        break
    print(row.FirstName)

Jerry
Dennis
Bart
Bruce
Bob


In [34]:
csr.execute('SELECT FirstName, LastName FROM Lab_Table')
while True:
    row = csr.fetchone()
    if not row:
        break
    print(row.FirstName, row.LastName)

Jerry Mouse
Dennis Bergkamp
Bart Simpson
Bruce Lee
Bob Marley


In [37]:
csr.execute('SELECT ID, FirstName, LastName FROM Lab_Table')
rows = csr.fetchall()
rows

[(2, 'Jerry', 'Mouse'),
 (4, 'Dennis', 'Bergkamp'),
 (6, 'Bart', 'Simpson'),
 (7, 'Bruce', 'Lee'),
 (8, 'Bob', 'Marley')]

In [38]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

2 Jerry Mouse
4 Dennis Bergkamp
6 Bart Simpson
7 Bruce Lee
8 Bob Marley


In [39]:
type(rows)

list

In [40]:
type(row.ID)

int

In [41]:
type(row.FirstName)

str

In [42]:
csr.execute("""SELECT ID, FirstName, LastName 
                FROM Lab_Table""")
rows = csr.fetchmany(5)
rows

[(2, 'Jerry', 'Mouse'),
 (4, 'Dennis', 'Bergkamp'),
 (6, 'Bart', 'Simpson'),
 (7, 'Bruce', 'Lee'),
 (8, 'Bob', 'Marley')]

In [45]:
df = pd.DataFrame(rows)
df

,0
0,"[2, Jerry, Mouse]"
1,"[4, Dennis, Bergkamp]"
2,"[6, Bart, Simpson]"
3,"[7, Bruce, Lee]"
4,"[8, Bob, Marley]"


In [47]:
row_list = []
for row in rows:
    row = list(row)
    row_list.append(row)
row_list    

[[2, 'Jerry', 'Mouse'],
 [4, 'Dennis', 'Bergkamp'],
 [6, 'Bart', 'Simpson'],
 [7, 'Bruce', 'Lee'],
 [8, 'Bob', 'Marley']]

In [48]:
columns = ['ID', 'FirstName', 'LastName']

In [50]:
df = pd.DataFrame(row_list, columns=columns)
df

,ID,FirstName,LastName
0,2,Jerry,Mouse
1,4,Dennis,Bergkamp
2,6,Bart,Simpson
3,7,Bruce,Lee
4,8,Bob,Marley


### *Connection BikeStores*

In [53]:
csr.execute('USE BikeStores')

In [55]:
query = """SELECT TABLE_NAME
           FROM INFORMATION_SCHEMA.TABLES
           WHERE TABLE_TYPE='BASE TABLE'
        """
csr.execute(query).fetchall()

[('categories', ),
 ('brands', ),
 ('products', ),
 ('customers', ),
 ('stores', ),
 ('staffs', ),
 ('orders', ),
 ('order_items', ),
 ('stocks', ),
 ('sysdiagrams', )]

In [56]:
csr.execute("""select * from production.brands""").fetchall()

[(1, 'Electra'),
 (2, 'Haro'),
 (3, 'Heller'),
 (4, 'Pure Cycles'),
 (5, 'Ritchey'),
 (6, 'Strider'),
 (7, 'Sun Bicycles'),
 (8, 'Surly'),
 (9, 'Trek')]

In [57]:
pd.read_sql("""select * from production.brands""", con=conn)

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek


In [59]:
df_prod = pd.read_sql("""select * from production.products""", con=conn)
df_prod

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
...,...,...,...,...,...,...
316,317,Trek Checkpoint ALR 5 - 2019,9,7,2019,1999.99
317,318,Trek Checkpoint ALR 5 Women's - 2019,9,7,2019,1999.99
318,319,Trek Checkpoint SL 5 Women's - 2019,9,7,2019,2799.99
319,320,Trek Checkpoint SL 6 - 2019,9,7,2019,3799.99


In [60]:
import sqlalchemy
import urllib

In [63]:
csr.execute('USE Lab_DB')

In [64]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'VAIO'
database = 'Lab_DB'
user = 'sa'
password = '14531453'

In [65]:
params = 'DRIVER='+driver + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + user + ';PWD=' + password

In [66]:
db_params = urllib.parse.quote_plus(params)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [67]:
df_prod.to_sql("products_bikes", engine, index=False, if_exists="append", schema="dbo")

In [68]:
df_new = pd.read_sql("""SELECT * FROM [dbo].[products_bikes] """,con = conn)
df_new.head()

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99
